<a href="https://colab.research.google.com/github/carolinampessoa/TechChallengeFase2/blob/main/TechChallengeFase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1) Definindo o problema**

Escolha um problema real que possa ser resolvido por meio de otimização genética. Descreva o problema, os objetivos e os critérios de sucesso.

##**2) Implementação do AG**

### 2.1) Importando bibiliotecas e definindo parâmetros iniciais

In [51]:
# Importação de bibliotecas
import random
import matplotlib.pyplot as plt

# Configurações iniciais do problema
NUM_TAREFAS = 1000
NUM_MAQUINAS = 20
TAMANHO_POPULACAO = 100
GERACOES = 200
TAXA_CROSSOVER = 0.8
TAXA_MUTACAO = 0.1

# Geração de tarefas aleatórias com tempo variando entre 1 e 15 minutos
tarefas = [random.randint(1, 15) for _ in range(NUM_TAREFAS)]

### 2.2) Codificação do indivíduo

No problema proposto, um indivíduo é representado por um vetor de valores inteiros (codificação real), onde cada elemento i representa a máquina em que foi alocada a tarefa i.

In [52]:
# CRIAR UM INDIVÍDUO (alocação de tarefas em máquinas)
def criar_individuo():
    return [random.randint(0, NUM_MAQUINAS - 1) for _ in range(NUM_TAREFAS)]

Exemplo de indíviduo gerado:

In [53]:
print(criar_individuo())

[6, 3, 7, 8, 19, 19, 12, 10, 1, 10, 0, 17, 14, 2, 12, 3, 2, 4, 7, 4, 7, 17, 5, 1, 18, 15, 12, 12, 2, 9, 13, 0, 1, 12, 7, 10, 0, 7, 10, 19, 14, 11, 10, 8, 4, 16, 11, 0, 19, 5, 2, 3, 18, 17, 11, 12, 13, 6, 19, 14, 11, 1, 8, 3, 15, 11, 4, 12, 15, 11, 3, 15, 9, 9, 15, 2, 1, 15, 2, 4, 17, 12, 8, 2, 17, 10, 5, 9, 0, 7, 16, 12, 4, 10, 19, 14, 6, 2, 4, 0, 11, 6, 7, 19, 14, 0, 9, 2, 7, 5, 8, 5, 0, 12, 18, 15, 15, 14, 4, 15, 3, 19, 6, 17, 10, 1, 3, 1, 3, 16, 5, 8, 12, 0, 7, 15, 15, 3, 0, 19, 14, 11, 5, 2, 6, 18, 0, 1, 11, 6, 16, 7, 13, 5, 11, 18, 17, 17, 19, 5, 10, 1, 15, 16, 14, 19, 18, 2, 17, 5, 19, 8, 1, 2, 7, 18, 16, 3, 1, 0, 18, 11, 13, 6, 14, 9, 19, 17, 16, 12, 19, 11, 18, 10, 13, 8, 10, 17, 17, 4, 14, 10, 16, 0, 17, 17, 7, 3, 19, 11, 4, 1, 8, 14, 6, 16, 0, 14, 9, 16, 1, 16, 18, 17, 14, 10, 6, 7, 17, 3, 19, 15, 4, 15, 18, 19, 2, 8, 17, 16, 18, 8, 5, 17, 2, 13, 1, 1, 10, 17, 5, 15, 15, 4, 11, 15, 10, 5, 4, 0, 15, 19, 11, 18, 14, 17, 2, 17, 11, 17, 3, 11, 15, 10, 1, 13, 11, 17, 17, 16, 17, 1

### 2.3) Função Fitness (makespan)

In [54]:

# AVALIAÇÃO CLÁSSICA (considera somente makespan)
def avaliar_classico(individuo):
    maquinas = [0] * NUM_MAQUINAS
    for i, m in enumerate(individuo):
        maquinas[m] += tarefas[i]
    return max(maquinas)

# AVALIAÇÃO MELHORADA (considera makespan + desbalanceamento)
def avaliar_melhorado(individuo):
    maquinas = [0] * NUM_MAQUINAS
    for i, m in enumerate(individuo):
        maquinas[m] += tarefas[i]
    makespan = max(maquinas)
    media = sum(maquinas) / NUM_MAQUINAS
    desvio = sum(abs(x - media) for x in maquinas) / NUM_MAQUINAS
    return makespan + 0.1 * desvio


### 2.4) Operadores genéticos (seleção, cruzamento e mutação)

In [55]:
# SELEÇÃO POR TORNEIO
def selecao(populacao, func_avaliar):
    a, b = random.sample(populacao, 2)
    return a if func_avaliar(a) < func_avaliar(b) else b

# CRUZAMENTO (crossover de um ponto)
def crossover(pai1, pai2):
    if random.random() < TAXA_CROSSOVER:
        ponto = random.randint(1, NUM_TAREFAS - 1)
        return pai1[:ponto] + pai2[ponto:]
    return pai1[:]


# CROSSOVER DE DOIS PONTOS
def aplicar_crossover(pai1, pai2, usar_crossover):
    if usar_crossover and random.random() < TAXA_CROSSOVER:
        p1, p2 = sorted(random.sample(range(NUM_TAREFAS), 2))
        return pai1[:p1] + pai2[p1:p2] + pai1[p2:]
    return pai1[:]

# MUTAÇÃO GUIADA
def aplicar_mutacao(individuo, usar_mutacao=True, guiada=False):
    if not usar_mutacao:
        return individuo
    num_mutacoes = int(NUM_TAREFAS * TAXA_MUTACAO)
    for _ in range(num_mutacoes):
        idx = random.randint(0, NUM_TAREFAS - 1)
        if guiada:
            cargas = [0] * NUM_MAQUINAS
            for i, m in enumerate(individuo):
                cargas[m] += tarefas[i]
            melhor_maquina = cargas.index(min(cargas))
            individuo[idx] = melhor_maquina
        else:
            individuo[idx] = random.randint(0, NUM_MAQUINAS - 1)
    return individuo

# MUTAÇÃO (muda a máquina de uma tarefa aleatória)
def mutacao(individuo):
    if random.random() < TAXA_MUTACAO:
        idx = random.randint(0, NUM_TAREFAS - 1)
        individuo[idx] = random.randint(0, NUM_MAQUINAS - 1)
    return individuo


In [56]:
# GREEDY SOLUTION (HOTSTART): aloca sempre na máquina mais livre
def solucao_greedy():
    maquinas = [0] * NUM_MAQUINAS
    individuo = []
    for t in tarefas:
        idx = maquinas.index(min(maquinas))
        maquinas[idx] += t
        individuo.append(idx)
    return individuo

# POPULAÇÃO INICIAL (com ou sem hotstart)
def criar_populacao(usar_injecao_greedy):
    pop = [criar_individuo() for _ in range(TAMANHO_POPULACAO - 5)] if usar_injecao_greedy else [criar_individuo() for _ in range(TAMANHO_POPULACAO)]
    if usar_injecao_greedy:
        base = solucao_greedy()
        for _ in range(5):
            mutado = aplicar_mutacao(base[:], usar_mutacao=True, guiada=True)
            pop.append(mutado)
    return pop


### 2.5) Loop principal do AG

In [57]:
# EXECUÇÃO DO AG COM CONFIGURAÇÕES DINÂMICAS
def executar_ag(nome_execucao, usar_crossover, usar_mutacao, melhorado=False):
    print(f"\n--- {nome_execucao} ---")
    usar_injecao_greedy = melhorado
    func_avaliar = avaliar_melhorado if melhorado else avaliar_classico
    populacao = criar_populacao(usar_injecao_greedy)
    melhor = min(populacao, key=func_avaliar)
    historico = [func_avaliar(melhor)]

    for geracao in range(GERACOES):
        nova_pop = [melhor]  # elitismo
        while len(nova_pop) < TAMANHO_POPULACAO:
            p1 = selecao(populacao, func_avaliar)
            p2 = selecao(populacao, func_avaliar)
            filho = aplicar_crossover(p1, p2, usar_crossover)
            filho = aplicar_mutacao(filho, usar_mutacao, guiada=melhorado)
            nova_pop.append(filho)

        populacao = nova_pop
        melhor_atual = min(populacao, key=func_avaliar)
        if func_avaliar(melhor_atual) < func_avaliar(melhor):
            melhor = melhor_atual

        historico.append(func_avaliar(melhor))
        print(f"Geração {geracao+1:3}: Melhor fitness = {func_avaliar(melhor)}")

    return melhor, historico, func_avaliar(melhor)


## **3) Testes e Resultados**

Avaliando resultados do AG com diferentes configurações

In [ ]:
# VARIAÇÕES (nome, crossover, mutação, versão melhorada)
versoes = [
    ("AG Clássico - sem crossover e sem mutação", False, False, False),
    ("AG Clássico - com mutação, sem crossover", False, True, False),
    ("AG Clássico - com crossover, sem mutação", True, False, False),
    ("AG Clássico - com crossover e mutação", True, True, False),
    ("AG Melhorado - sem crossover e sem mutação", False, False, True),
    ("AG Melhorado - com mutação, sem crossover", False, True, True),
    ("AG Melhorado - com crossover, sem mutação", True, False, True),
    ("AG Melhorado - com crossover e mutação", True, True, True),
]

resultados = {}
for nome, cross, mut, melhorado in versoes:
    melhor, hist, fitness = executar_ag(nome, cross, mut, melhorado)
    resultados[nome] = {
        "historico": hist,
        "fitness": fitness,
    }

# Greedy Solution
sol_greedy = solucao_greedy()
fitness_greedy = avaliar_classico(sol_greedy)

print("\n==================== RESULTADOS FINAIS ====================")
for nome, dados in resultados.items():
    print(f"{nome:50} -> Fitness final: {dados['fitness']:.2f}")
print(f"{'Greedy Solution':50} -> Fitness final: {fitness_greedy:.2f}")
print("===========================================================")




--- AG Clássico - sem crossover e sem mutação ---
Geração   1: Melhor fitness = 472
Geração   2: Melhor fitness = 472
Geração   3: Melhor fitness = 472
Geração   4: Melhor fitness = 472
Geração   5: Melhor fitness = 472
Geração   6: Melhor fitness = 472
Geração   7: Melhor fitness = 472
Geração   8: Melhor fitness = 472
Geração   9: Melhor fitness = 472
Geração  10: Melhor fitness = 472
Geração  11: Melhor fitness = 472
Geração  12: Melhor fitness = 472
Geração  13: Melhor fitness = 472
Geração  14: Melhor fitness = 472
Geração  15: Melhor fitness = 472
Geração  16: Melhor fitness = 472
Geração  17: Melhor fitness = 472
Geração  18: Melhor fitness = 472
Geração  19: Melhor fitness = 472
Geração  20: Melhor fitness = 472
Geração  21: Melhor fitness = 472
Geração  22: Melhor fitness = 472
Geração  23: Melhor fitness = 472
Geração  24: Melhor fitness = 472
Geração  25: Melhor fitness = 472
Geração  26: Melhor fitness = 472
Geração  27: Melhor fitness = 472
Geração  28: Melhor fitness = 4

Gráfico comparativo das diferentes versões de Algoritmo Genético implementadas.

In [ ]:
# GRÁFICO COMPARATIVO
plt.figure(figsize=(14, 7))
for nome, dados in resultados.items():
    plt.plot(dados["historico"], label=nome)
plt.axhline(y=fitness_greedy, color='black', linestyle='--', label="Greedy Solution")
plt.xlabel("Geração")
plt.ylabel("Fitness")
plt.title("Comparação entre variações de Algoritmos Genéticos")
plt.legend(loc='upper right', fontsize=8)
plt.grid(True)
plt.tight_layout()
plt.show()